<a href="https://colab.research.google.com/github/gsc16/MachineLearning/blob/master/hw1q3.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

Imports

In [1]:
import keras
import numpy as np
from keras.datasets import mnist
import math

Using TensorFlow backend.


Loading images from MNIST dataset

In [0]:
(train_images_original, train_labels_original), (test_images_original, test_labels_original) = mnist.load_data()

Displaying the original values

In [3]:
train_images_original.shape, train_labels_original, test_images_original.shape, test_labels_original.shape

((60000, 28, 28),
 array([5, 0, 4, ..., 5, 6, 8], dtype=uint8),
 (10000, 28, 28),
 (10000,))

Extracting images - Flattening and Reshaping

In [0]:
train_images_new = train_images_original.reshape(60000, 28*28)
train_images_new = train_images_new.astype('float32') / 255
train_images_new = train_images_new.T

test_images_new = test_images_original.reshape(10000, 28*28)
test_images_new = test_images_new.astype('float32') / 255
test_images_new = test_images_new.T


Converting integer vectors to binary

In [0]:
# Function to convert integer values of labels to binary
def convert_to_binary(train_labels_original):
    op = []
    for i in train_labels_original:
      t = np.zeros(10) # 10 is the number of classifiers
      t[i] = 1
      op.append(t)
    op = np.array(op)
    op = op.reshape(train_labels_original.shape[0],10)
    return op

In [0]:
train_labels_new = convert_to_binary(train_labels_original)
test_labels_new = convert_to_binary(test_labels_original)
train_labels_new = train_labels_new.T
test_labels_new = test_labels_new.T

Displaying new values

In [7]:
train_images_new.shape, train_labels_new.shape, test_images_new.shape, test_labels_new.shape

((784, 60000), (10, 60000), (784, 10000), (10, 10000))

Initializing the learning rate, batch size and number of epochs

In [0]:
batch_size = 32
num_classes = 10
epochs = 12
learning_rate = 0.001

Definitions

In [0]:
#Function to calculate logistic regression
def log_reg(w,b,X):
  return np.dot(w, X) + b

#Function to calculate the value of softmax function
def softmax(x): 
  e_x = np.exp(x - np.max(x))
  return e_x/e_x.sum(axis=0)

# Function to calculate the mini batches
def cal_mini_batches(X, Y):
    for index in range(0, X.shape[0] - batch_size + 1, batch_size):
        new_batch = slice(index, index + batch_size)
        yield X[new_batch], Y[new_batch]
        
# Function to calculate cross entropy loss
def cross_entropy(Y, Y_Calc):
    return -np.mean(Y * np.log(Y_Calc))

Training the classifiers

In [10]:
#initialization of weights
w = np.random.randn(train_labels_new.shape[0], train_images_new.shape[0]) * 0.01
b = np.zeros(shape=(train_labels_new.shape[0], 1))
  
#training
for epoch in range(epochs):
     
  #splitting the training and test  set according to bacth sizes
  for batch in cal_mini_batches(train_images_new.T, train_labels_new.T):
 
    x_batch, y_batch = batch
    xt = x_batch.T
    yt = y_batch.T
    m = xt.shape[0]
        
    #forward propogation
    Y = log_reg(w,b,xt)
    Z = softmax(Y);

    cost = cross_entropy(yt, Z)
        
    #back propogation - gradient descent
    loss = Z-yt

    gradient_w = (1/m) * np.dot(loss, xt.T)
    gradient_b = (1/m) * np.sum(loss, axis=1, keepdims=True)

    w = w - learning_rate * gradient_w
    b = b - learning_rate * gradient_b
    

# Creating one hot encoding of the softmax ouptut to calculate the accuracy
y_train = log_reg(w,b,train_images_new)
z_train = softmax(y_train)
z_train  =  z_train.T

y_test = log_reg(w,b,test_images_new)
z_test = softmax(y_test)
z_test  =  z_test.T
  
train_labels_new = train_labels_new.T
test_labels_new  = test_labels_new.T
  
train_labels_prediction = np.zeros_like(train_labels_new)
train_labels_prediction[np.arange(len(z_train)), z_train.argmax(1)] = 1
  
test_labels_prediction = np.zeros_like(test_labels_new)
test_labels_prediction[np.arange(len(z_test)), z_test.argmax(1)] = 1
  
  
# Calculating training and testing accuracies
train_acc = 100 - np.mean(np.abs(train_labels_prediction - train_labels_new)) * 100
train_acc = np.round(train_acc,2)
test_acc = 100 - np.mean(np.abs(test_labels_prediction - test_labels_new)) * 100
test_acc = np.round(test_acc,2)
print("Training accuracy is "+ str(train_acc)+" %")
print("Testing accuracy is "+str(test_acc)+" %")

Training accuracy is 95.47 %
Testing accuracy is 95.65 %
